In [1]:
import keras
import numpy as np
from keras.models import Sequential,Model
from keras.layers import Dense,Dropout,LSTM,Input,Bidirectional
from sklearn.model_selection import cross_val_score 
from nltk.tokenize import word_tokenize,sent_tokenize
from ast import literal_eval
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [2]:
import keras
import numpy as np
from keras.layers import *
from sklearn.model_selection import cross_val_score 

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Remove the rows which contains values ['No resposne']

In [4]:
import pandas as pd
df=pd.read_csv('/content/drive/My Drive/text_emotion.csv')
print(len(df))

40000


In [5]:
df=df.drop(['tweet_id','author'], axis = 1) 
import re
df['content'] = df['content'].str.replace(re.compile('http\S+'),'')
df['content'] = df['content'].str.replace(re.compile('@\w+'),'')
# Removing words which starts with '#'(representing any trend) in the tweets

df['content'] = df['content'].str.replace(re.compile('#\w+'),'')
# phrases cleaning & punctuation removal
def sentence_cleaning(sentence):
    sentence=re.sub('\'d',' would',sentence)
    sentence=re.sub('\'ll',' will',sentence)
    sentence=re.sub('\'ve',' have',sentence)
    sentence=re.sub('\'s',' is',sentence)
    sentence=re.sub('n\'t',' not',sentence)
    sentence=re.sub("won't",' will not',sentence)
    sentence=re.sub("can't",' cannot',sentence)
    sentence=re.sub("ain't",' am not',sentence)
    sentence=re.sub('\W',' ',sentence) #'\W' to remove all non-alphanumeric characters(punctuations)
    sentence=sentence.lower()
    return sentence
df['content'] = df['content'].apply(sentence_cleaning)
df.head()


,sentiment,content
0,empty,i know i was listenin to bad habit earlier a...
1,sadness,layin n bed with a headache ughhhh waitin o...
2,sadness,funeral ceremony gloomy friday
3,enthusiasm,wants to hang out with friends soon
4,neutral,we want to trade with someone who has houston...


In [6]:
# We are going to work on only five  emotions i.e happiness, worry,sadness,surprise,anger.
index_names = df[ df['sentiment'] == 'empty' ].index 
df.drop(index_names, inplace = True) 

In [7]:
index_names = df[ df['sentiment'] == 'boredom' ].index 
df.drop(index_names, inplace = True) 

In [8]:
index_names = df[ df['sentiment'] == 'enthusiasm' ].index 
df.drop(index_names, inplace = True) 

In [9]:
index_names = df[ df['sentiment'] == 'fun' ].index 
df.drop(index_names, inplace = True) 

In [10]:
index_names = df[ df['sentiment'] == 'hate' ].index 
df.drop(index_names, inplace = True) 

In [11]:
index_names = df[ df['sentiment'] == 'neutral' ].index 
df.drop(index_names, inplace = True) 

In [12]:
index_names = df[ df['sentiment'] == 'love' ].index 
df.drop(index_names, inplace = True) 

In [13]:
index_names = df[ df['sentiment'] == 'relief' ].index 
df.drop(index_names, inplace = True) 

In [14]:
# Make Own Index as index 
# In this case default index is exist  
df.reset_index(inplace = True) 

In [15]:
texts=df['content']
print(texts)

0        layin n bed with a headache  ughhhh   waitin o...
1                      funeral ceremony   gloomy friday   
2        re pinging   why did not you go to prom  bc my...
3        i should be sleep  but im not  thinking about ...
4                                           hmmm   is down
                               ...                        
21125    going to watch boy in the striped pj is hope i...
21126    gave the bikes a thorough wash  degrease it an...
21127    had such and amazing time last night  mcfly we...
21128                        succesfully following tayla  
21129     wassup beautiful    follow me    peep out my ...
Name: content, Length: 21130, dtype: object


In [16]:
texts=[word_tokenize(sent) for sent in texts]
print(texts[0])

['layin', 'n', 'bed', 'with', 'a', 'headache', 'ughhhh', 'waitin', 'on', 'your', 'call']


In [17]:
def padd(arr):
    for i in range(50-len(arr)):
        arr.append('<pre>')
    return arr[:50]
for i in range(len(texts)):
  texts[i]=padd(texts[i])

In [18]:
vocab_f=open('/content/drive/My Drive/glove.6B.50d.txt')
word_to_emb={}
for line in vocab_f:
  word_to_emb[line.split()[0]]=[float(i) for i in line.split()[1:]] 
word_to_emb['<pre>']=[0]*50

In [19]:
embedded_arr=[] 
for each_sentence in texts:
    embedded_arr.append([])
    for word in each_sentence:
        if word.lower() in word_to_emb:
            embedded_arr[-1].append(word_to_emb[word.lower()])
        else:
            embedded_arr[-1].append([0]*50)
print(embedded_arr[0])

[[0.062774, 0.25423, 0.54447, -0.49013, -0.48846, 0.7485, -0.172, 0.35337, -0.19946, 1.1631, -0.47937, 0.27788, -0.7734, 1.027, -0.78503, -0.50238, 0.11522, -0.92131, 0.16118, 0.089774, -0.22911, -0.23128, -0.1429, -0.45046, 0.38843, 0.39312, 0.24965, 0.34096, 1.0334, -0.0048145, -0.65535, 0.42406, -1.0536, 1.3787, 0.20906, -0.18762, 0.247, 0.024948, 0.59644, 0.34294, -0.061858, -0.65872, -1.2111, 0.24885, -0.027022, -0.78659, 1.2393, 0.37686, 0.89725, -0.21693], [-0.27004, 1.1144, 1.0493, 0.57924, 0.78968, 0.14137, 1.4291, -1.0101, -0.78908, -0.27072, -0.21101, 0.29724, -0.37254, -0.31723, -0.51419, 0.35462, 0.35406, -0.093878, -0.25165, -0.62402, -0.064895, -0.97966, 1.8001, 0.95118, -0.69039, 0.2324, -0.55899, 0.16657, 0.33153, 0.023364, 2.0653, -0.45236, -0.67552, 0.34039, 0.60026, -1.1793, 0.88805, -0.92985, 0.020351, 0.68063, 0.85255, -0.097895, -0.68661, -0.2147, -0.70778, 0.35091, 0.31743, -0.77324, 0.42681, 1.0782], [0.75344, 0.96383, 0.10188, -0.67978, 0.38461, 0.027779, -0.3

In [20]:
X=np.array(embedded_arr)
print(np.shape(X))

(21130, 50, 50)


In [21]:
y = df['sentiment'].unique()
print(y)

['sadness' 'worry' 'surprise' 'happiness' 'anger']


In [22]:
y_map = {} 
for i in range(len(y)):
  y_map[y[i]] = i

print(y_map)

{'sadness': 0, 'worry': 1, 'surprise': 2, 'happiness': 3, 'anger': 4}


In [23]:
df.sentiment = df.sentiment.map(y_map)

In [24]:
# Generating one hot encoded data for the sentiment from the above labeled data
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(handle_unknown='ignore')
res = enc.fit_transform(df[['sentiment']]).toarray()
print(res.shape)


(21130, 5)


In [25]:
Y= res
print(Y)

[[1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 ...
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0.]]


In [26]:
from keras.layers import Embedding
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.1, random_state=42)

In [27]:
def model1(X,Y,input_size1,input_size2,output_size):
  m=Sequential()
  m.add(Bidirectional(LSTM(100,input_shape=(input_size1,input_size2))))
  m.add(Dropout(0.5))
  m.add(Dense(output_size,activation='softmax'))
  m.compile('Adam','categorical_crossentropy',['accuracy'])
  m.fit(X,Y,epochs=10, batch_size=128)
  return m

In [28]:
bilstmModel=model1(X_train,Y_train,50,50,5)

Epoch 1/10
149/149 [==============================] - 10s 9ms/step - loss: 1.3468 - accuracy: 0.3993
Epoch 2/10
149/149 [==============================] - 1s 9ms/step - loss: 1.2280 - accuracy: 0.4724
Epoch 3/10
149/149 [==============================] - 1s 10ms/step - loss: 1.1949 - accuracy: 0.4794
Epoch 4/10
149/149 [==============================] - 1s 9ms/step - loss: 1.1764 - accuracy: 0.4968
Epoch 5/10
149/149 [==============================] - 1s 8ms/step - loss: 1.1669 - accuracy: 0.4938
Epoch 6/10
149/149 [==============================] - 1s 8ms/step - loss: 1.1584 - accuracy: 0.5019
Epoch 7/10
149/149 [==============================] - 1s 9ms/step - loss: 1.1425 - accuracy: 0.5103
Epoch 8/10
149/149 [==============================] - 1s 8ms/step - loss: 1.1399 - accuracy: 0.5056
Epoch 9/10
149/149 [==============================] - 1s 8ms/step - loss: 1.1248 - accuracy: 0.5137
Epoch 10/10
149/149 [==============================] - 1s 9ms/step - loss: 1.1101 - accuracy: 0.52

In [29]:
bilstmModel.evaluate(X_test,Y_test)

67/67 [==============================] - 1s 6ms/step - loss: 1.1730 - accuracy: 0.4950


[1.1730314493179321, 0.4950307607650757]

In [30]:
y_inverse_map={}
for i in y_map:
  y_inverse_map[y_map[i]]=i
print(y_inverse_map)

{0: 'sadness', 1: 'worry', 2: 'surprise', 3: 'happiness', 4: 'anger'}


In [31]:
def prepare(s):
  s=word_tokenize(s)
  #print("s= ",s)
  k=padd(s)
  embedded_arr=[]
  embedded_arr.append([])
  for word in k:
        if word.lower() in word_to_emb:
            embedded_arr[-1].append(word_to_emb[word.lower()])
        else:
            embedded_arr[-1].append([0]*50)
  x_input=np.array(embedded_arr)
  #print(np.shape(x_input))
  arr = bilstmModel.predict(x_input)
  return arr.argmax()

In [49]:
s=input()
emotion=prepare(s)
print(y_inverse_map[emotion])

what a sunny day!
happiness


In [50]:
bilstmModel.save('lstm_model.h5')